In [1]:
# if (!requireNamespace("BiocManager", quietly=TRUE)) 
#     install.packages("BiocManager") 

# BiocManager::install("ANCOMBC")

In [1]:
.libPaths(R.home('/opt/homebrew/Caskroom/miniforge/base/lib/R/library'))
Output = '/Users/alexis/Library/CloudStorage/OneDrive-UniversityofNorthCarolinaatChapelHill/CEMALB_DataAnalysisPM/Projects/P1013. Microbiome Peanut Allergy/P1013.3. Analyses/P1013.3.2. Descriptive Statistics/Output'
cur_date = '090525'

library(readxl)
library(openxlsx)
library(writexl)
library(tidyverse)
library(factoextra)
library(gtsummary)
library(ANCOMBC) 

# reading in files
saliva_df = data.frame(read_excel("Input/Processed_Saliva_Data_090425.xlsx")) %>%
    mutate(Age = as.numeric(Age), IgE = as.numeric(IgE)) %>%
    # removing those failed the IgE protein
    filter(IgE_Binarized != 'Fail')
stool_df = data.frame(read_excel("Input/Processed_Stool_Data_090425.xlsx")) %>%
    mutate(Age = as.numeric(Age), IgE = as.numeric(IgE)) %>%
    filter(IgE_Binarized != 'Fail')

── Attaching core tidyverse packages ────────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.2     ✔ tibble    3.3.0
✔ lubridate 1.9.4     ✔ tidyr     1.3.1
✔ purrr     1.1.0     
── Conflicts ──────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
Welcome! Want to learn more? See two factoextra-related books at https://goo.gl/ve3WBa



In [2]:
head(saliva_df)

,Original_Sample_ID,Subject_No,Sample_ID,Time_Point,Treatment,Outcome,Age,IgE,IgE_Binarized,k__Bacteria.__.__.__.__.__.__,⋯,k__Bacteria.p__Proteobacteria.c__Gammaproteobacteria.o__Pasteurellales.f__Pasteurellaceae.__.__,k__Bacteria.p__Proteobacteria.c__Gammaproteobacteria.o__Pasteurellales.f__Pasteurellaceae.g__Aggregatibacter.__,k__Bacteria.p__Proteobacteria.c__Gammaproteobacteria.o__Pasteurellales.f__Pasteurellaceae.g__Aggregatibacter.s__aphrophilus,k__Bacteria.p__Proteobacteria.c__Gammaproteobacteria.o__Pasteurellales.f__Pasteurellaceae.g__Haemophilus.__,k__Bacteria.p__Proteobacteria.c__Gammaproteobacteria.o__Pasteurellales.f__Pasteurellaceae.g__Haemophilus.s__parainfluenzae,k__Bacteria.p__Saccharibacteria_.TM7..c__Saccharibacteria_.TM7._.C.1..o__Saccharibacteria_.TM7._.O.1..f__Saccharibacteria_.TM7._.F.1..g__Saccharibacteria_.TM7._.G.1..__,k__Bacteria.p__Saccharibacteria_.TM7..c__Saccharibacteria_.TM7._.C.1..o__Saccharibacteria_.TM7._.O.1..f__Saccharibacteria_.TM7._.F.1..g__Saccharibacteria_.TM7._.G.1..s__bacterium_HMT_347,k__Bacteria.p__Saccharibacteria_.TM7..c__Saccharibacteria_.TM7._.C.1..o__Saccharibacteria_.TM7._.O.1..f__Saccharibacteria_.TM7._.F.1..g__Saccharibacteria_.TM7._.G.1..s__bacterium_HMT_352,k__Bacteria.p__Saccharibacteria_.TM7..c__Saccharibacteria_.TM7._.C.1..o__Saccharibacteria_.TM7._.O.1..f__Saccharibacteria_.TM7._.F.1..g__Saccharibacteria_.TM7._.G.3..s__bacterium_HMT_351,k__Bacteria.p__Saccharibacteria_.TM7..c__Saccharibacteria_.TM7._.C.1..o__Saccharibacteria_.TM7._.O.1..f__Saccharibacteria_.TM7._.F.1..g__Saccharibacteria_.TM7._.G.6..s__bacterium_HMT_870
,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,FS01baseline,1,Pre_1,Pre,P,F,3.838356,41.00,High,7,⋯,57,13,0,213,775,0,14,246,44,64
2,FS0236mo,2,Post_2,Post,A,DS,1.873973,14.20,Low,63,⋯,698,0,43,98,1657,0,0,423,146,123
3,FS02baseline,2,Pre_2,Pre,A,DS,1.873973,14.20,Low,0,⋯,0,0,46,371,3359,0,0,0,0,112
4,FS0436mo,3,Post_3,Post,P,F,3.715068,67.40,High,34,⋯,0,0,0,0,92,0,200,67,32,201
5,FS04baseline,3,Pre_3,Pre,P,F,3.715068,67.40,High,4,⋯,0,0,0,0,43,414,47,0,2,1050
6,FS0536mo,4,Post_4,Post,A,DS,3.991781,9.59,Low,6,⋯,0,17,0,139,2521,0,0,502,39,101


# Summary Statistics

Determining if there are any significant differences based on outcome.

In [3]:
saliva_df$Treatment = factor(saliva_df$Treatment, levels = c('A', 'P'))
stool_df$Treatment = factor(stool_df$Treatment, levels = c('A', 'P'))
saliva_df$IgE_Binarized = factor(saliva_df$IgE_Binarized, levels = c('Low', 'High'))
stool_df$IgE_Binarized = factor(stool_df$IgE_Binarized, levels = c('Low', 'High'))
saliva_df$Outcome = factor(saliva_df$Outcome, levels = c('F', 'DS','SU'))
stool_df$Outcome = factor(stool_df$Outcome, levels = c('F', 'DS','SU'))

In [4]:
saliva_df %>%
    filter(Time_Point == 'Pre') %>%
    tbl_summary(by = Outcome, missing = "no", 
    include = colnames(saliva_df[c(5,7:9)]), 
              statistic = list(all_continuous() ~ "{mean} ({sd})",
                               all_categorical() ~ "{n} ({p}%)")) %>%
    add_n() %>% 
    add_p(test = list(#all_continuous() ~ "ancova.test",
                    all_categorical() ~ "fisher.test")) %>% # adding p value from anova
    as_tibble() #%>%
    #write_xlsx(., "Output/TableX.xlsx")

**Characteristic**,**N**,**F** N = 23,**DS** N = 4,**SU** N = 10,**p-value**
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
Treatment,37,NA,NA,NA,<0.001
A,NA,5 (22%),4 (100%),10 (100%),NA
P,NA,18 (78%),0 (0%),0 (0%),NA
Age,36,2.61 (0.99),2.60 (1.11),2.36 (1.05),0.7
IgE,35,47 (51),46 (73),14 (35),0.009
IgE_Binarized,36,NA,NA,NA,0.002
Low,NA,9 (41%),3 (75%),10 (100%),NA
High,NA,13 (59%),1 (25%),0 (0%),NA


In [5]:
stool_df %>%
    filter(Time_Point == 'Pre') %>%
    tbl_summary(by = Outcome, missing = "no", 
    include = colnames(saliva_df[c(5,7:9)]), 
              statistic = list(all_continuous() ~ "{mean} ({sd})",
                               all_categorical() ~ "{n} ({p}%)")) %>%
    add_n() %>% 
    add_p(test = list(#all_continuous() ~ "ancova.test",
                    all_categorical() ~ "fisher.test")) %>% # adding p value from anova
    as_tibble() #%>%
    #write_xlsx(., "Output/TableX.xlsx")

**Characteristic**,**N**,**F** N = 22,**DS** N = 2,**SU** N = 10,**p-value**
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
Treatment,34,NA,NA,NA,<0.001
A,NA,4 (18%),2 (100%),10 (100%),NA
P,NA,18 (82%),0 (0%),0 (0%),NA
Age,34,2.44 (0.99),2.42 (0.78),2.15 (0.92),0.7
IgE,33,47 (51),85 (100),4 (4),<0.001
IgE_Binarized,33,NA,NA,NA,0.002
Low,NA,8 (38%),1 (50%),10 (100%),NA
High,NA,13 (62%),1 (50%),0 (0%),NA


In [6]:
saliva_df$Outcome = as.character(saliva_df$Outcome)
stool_df$Outcome = as.character(stool_df$Outcome)

combined_saliva_df = saliva_df  %>%
    # ended up reassinging 'DS' to 'F' due to small sample size
    mutate(Outcome = ifelse(Outcome == 'DS', 'F', Outcome))

combined_stool_df = stool_df  %>%
    # ended up reassinging 'DS' to 'F' due to small sample size
    mutate(Outcome = ifelse(Outcome == 'DS', 'F', Outcome))

combined_saliva_df$Outcome = factor(combined_saliva_df$Outcome, levels = c('F', 'SU'))
combined_stool_df$Outcome = factor(combined_stool_df$Outcome, levels = c('F', 'SU'))

head(combined_saliva_df)

,Original_Sample_ID,Subject_No,Sample_ID,Time_Point,Treatment,Outcome,Age,IgE,IgE_Binarized,k__Bacteria.__.__.__.__.__.__,⋯,k__Bacteria.p__Proteobacteria.c__Gammaproteobacteria.o__Pasteurellales.f__Pasteurellaceae.__.__,k__Bacteria.p__Proteobacteria.c__Gammaproteobacteria.o__Pasteurellales.f__Pasteurellaceae.g__Aggregatibacter.__,k__Bacteria.p__Proteobacteria.c__Gammaproteobacteria.o__Pasteurellales.f__Pasteurellaceae.g__Aggregatibacter.s__aphrophilus,k__Bacteria.p__Proteobacteria.c__Gammaproteobacteria.o__Pasteurellales.f__Pasteurellaceae.g__Haemophilus.__,k__Bacteria.p__Proteobacteria.c__Gammaproteobacteria.o__Pasteurellales.f__Pasteurellaceae.g__Haemophilus.s__parainfluenzae,k__Bacteria.p__Saccharibacteria_.TM7..c__Saccharibacteria_.TM7._.C.1..o__Saccharibacteria_.TM7._.O.1..f__Saccharibacteria_.TM7._.F.1..g__Saccharibacteria_.TM7._.G.1..__,k__Bacteria.p__Saccharibacteria_.TM7..c__Saccharibacteria_.TM7._.C.1..o__Saccharibacteria_.TM7._.O.1..f__Saccharibacteria_.TM7._.F.1..g__Saccharibacteria_.TM7._.G.1..s__bacterium_HMT_347,k__Bacteria.p__Saccharibacteria_.TM7..c__Saccharibacteria_.TM7._.C.1..o__Saccharibacteria_.TM7._.O.1..f__Saccharibacteria_.TM7._.F.1..g__Saccharibacteria_.TM7._.G.1..s__bacterium_HMT_352,k__Bacteria.p__Saccharibacteria_.TM7..c__Saccharibacteria_.TM7._.C.1..o__Saccharibacteria_.TM7._.O.1..f__Saccharibacteria_.TM7._.F.1..g__Saccharibacteria_.TM7._.G.3..s__bacterium_HMT_351,k__Bacteria.p__Saccharibacteria_.TM7..c__Saccharibacteria_.TM7._.C.1..o__Saccharibacteria_.TM7._.O.1..f__Saccharibacteria_.TM7._.F.1..g__Saccharibacteria_.TM7._.G.6..s__bacterium_HMT_870
,<chr>,<dbl>,<chr>,<chr>,<fct>,<fct>,<dbl>,<dbl>,<fct>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,FS01baseline,1,Pre_1,Pre,P,F,3.838356,41.00,High,7,⋯,57,13,0,213,775,0,14,246,44,64
2,FS0236mo,2,Post_2,Post,A,F,1.873973,14.20,Low,63,⋯,698,0,43,98,1657,0,0,423,146,123
3,FS02baseline,2,Pre_2,Pre,A,F,1.873973,14.20,Low,0,⋯,0,0,46,371,3359,0,0,0,0,112
4,FS0436mo,3,Post_3,Post,P,F,3.715068,67.40,High,34,⋯,0,0,0,0,92,0,200,67,32,201
5,FS04baseline,3,Pre_3,Pre,P,F,3.715068,67.40,High,4,⋯,0,0,0,0,43,414,47,0,2,1050
6,FS0536mo,4,Post_4,Post,A,F,3.991781,9.59,Low,6,⋯,0,17,0,139,2521,0,0,502,39,101


In [7]:
combined_saliva_df %>%
    filter(Time_Point == 'Pre') %>%
    tbl_summary(by = Outcome, missing = "no", 
    include = colnames(saliva_df[c(5,7:9)]), 
              statistic = list(all_continuous() ~ "{mean} ({sd})",
                               all_categorical() ~ "{n} ({p}%)")) %>%
    add_n() %>% 
    add_p(test = list(#all_continuous() ~ "ancova.test",
                    all_categorical() ~ "fisher.test")) %>% # adding p value from anova
    as_tibble() %>%
    write_xlsx(., "Output/Table_Saliva.xlsx")

The following warnings were returned during `add_p()`:
! For variable `Age` (`Outcome`) and "estimate", "statistic", "p.value",
  "conf.low", and "conf.high" statistics: cannot compute exact p-value with ties
! For variable `Age` (`Outcome`) and "estimate", "statistic", "p.value",
  "conf.low", and "conf.high" statistics: cannot compute exact confidence
  intervals with ties
! For variable `IgE` (`Outcome`) and "estimate", "statistic", "p.value",
  "conf.low", and "conf.high" statistics: cannot compute exact p-value with ties
! For variable `IgE` (`Outcome`) and "estimate", "statistic", "p.value",
  "conf.low", and "conf.high" statistics: cannot compute exact confidence
  intervals with ties


In [8]:
combined_stool_df %>%
    filter(Time_Point == 'Pre') %>%
    tbl_summary(by = Outcome, missing = "no", 
    include = colnames(saliva_df[c(5,7:9)]), 
              statistic = list(all_continuous() ~ "{mean} ({sd})",
                               all_categorical() ~ "{n} ({p}%)")) %>%
    add_n() %>% 
    add_p(test = list(#all_continuous() ~ "ancova.test",
                    all_categorical() ~ "fisher.test")) %>% # adding p value from anova
    as_tibble() %>%
    write_xlsx(., "Output/Table_Stool.xlsx")

The following warnings were returned during `add_p()`:
! For variable `IgE` (`Outcome`) and "estimate", "statistic", "p.value",
  "conf.low", and "conf.high" statistics: cannot compute exact p-value with ties
! For variable `IgE` (`Outcome`) and "estimate", "statistic", "p.value",
  "conf.low", and "conf.high" statistics: cannot compute exact confidence
  intervals with ties


# Descriptive Statistics

I used the [ANCOM-BC2](https://www.bioconductor.org/packages/release/bioc/vignettes/ANCOMBC/inst/doc/ANCOMBC2.html) package to perform a differential abundance analysis ([ref](https://www.nature.com/articles/s41592-023-02092-7#Sec1)) to run a couple of different models. 

## 1. How do baseline microbiome species differ across outcomes? How do these differences vary after controlling for age and IgE protein concentrations?

In [21]:
# making the row names the sample ids
baseline_saliva_df = combined_saliva_df %>%
    # only keep participants' baseline samples and those that didn't fail the IgE protein
    filter(Time_Point == 'Pre' & IgE_Binarized != 'Fail') %>%
    # making the row names the sample ids
    column_to_rownames(var = 'Sample_ID')

baseline_stool_df = combined_stool_df %>%
    filter(Time_Point == 'Pre' & IgE_Binarized != 'Fail') %>%
    column_to_rownames(var = 'Sample_ID')

baseline_saliva_df$Outcome = factor(baseline_saliva_df$Outcome, levels = c('F', 'SU'))
baseline_stool_df$Outcome = factor(baseline_stool_df$Outcome, levels = c('F', 'SU'))

head(baseline_saliva_df)

,Original_Sample_ID,Subject_No,Time_Point,Treatment,Outcome,Age,IgE,IgE_Binarized,k__Bacteria.__.__.__.__.__.__,k__Bacteria.p__Absconditabacteria_.SR1..c__Absconditabacteria_.SR1._.C.1..o__Absconditabacteria_.SR1._.O.1..f__Absconditabacteria_.SR1._.F.1..g__Absconditabacteria_.SR1._.G.1..s__bacterium_HMT_874,⋯,k__Bacteria.p__Proteobacteria.c__Gammaproteobacteria.o__Pasteurellales.f__Pasteurellaceae.__.__,k__Bacteria.p__Proteobacteria.c__Gammaproteobacteria.o__Pasteurellales.f__Pasteurellaceae.g__Aggregatibacter.__,k__Bacteria.p__Proteobacteria.c__Gammaproteobacteria.o__Pasteurellales.f__Pasteurellaceae.g__Aggregatibacter.s__aphrophilus,k__Bacteria.p__Proteobacteria.c__Gammaproteobacteria.o__Pasteurellales.f__Pasteurellaceae.g__Haemophilus.__,k__Bacteria.p__Proteobacteria.c__Gammaproteobacteria.o__Pasteurellales.f__Pasteurellaceae.g__Haemophilus.s__parainfluenzae,k__Bacteria.p__Saccharibacteria_.TM7..c__Saccharibacteria_.TM7._.C.1..o__Saccharibacteria_.TM7._.O.1..f__Saccharibacteria_.TM7._.F.1..g__Saccharibacteria_.TM7._.G.1..__,k__Bacteria.p__Saccharibacteria_.TM7..c__Saccharibacteria_.TM7._.C.1..o__Saccharibacteria_.TM7._.O.1..f__Saccharibacteria_.TM7._.F.1..g__Saccharibacteria_.TM7._.G.1..s__bacterium_HMT_347,k__Bacteria.p__Saccharibacteria_.TM7..c__Saccharibacteria_.TM7._.C.1..o__Saccharibacteria_.TM7._.O.1..f__Saccharibacteria_.TM7._.F.1..g__Saccharibacteria_.TM7._.G.1..s__bacterium_HMT_352,k__Bacteria.p__Saccharibacteria_.TM7..c__Saccharibacteria_.TM7._.C.1..o__Saccharibacteria_.TM7._.O.1..f__Saccharibacteria_.TM7._.F.1..g__Saccharibacteria_.TM7._.G.3..s__bacterium_HMT_351,k__Bacteria.p__Saccharibacteria_.TM7..c__Saccharibacteria_.TM7._.C.1..o__Saccharibacteria_.TM7._.O.1..f__Saccharibacteria_.TM7._.F.1..g__Saccharibacteria_.TM7._.G.6..s__bacterium_HMT_870
,<chr>,<dbl>,<chr>,<chr>,<fct>,<dbl>,<dbl>,<fct>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Pre_1,FS01baseline,1,Pre,P,F,3.838356,41.00,High,7,30,⋯,57,13,0,213,775,0,14,246,44,64
Pre_2,FS02baseline,2,Pre,A,F,1.873973,14.20,Low,0,16,⋯,0,0,46,371,3359,0,0,0,0,112
Pre_3,FS04baseline,3,Pre,P,F,3.715068,67.40,High,4,0,⋯,0,0,0,0,43,414,47,0,2,1050
Pre_4,FS05baseline,4,Pre,A,F,3.991781,9.59,Low,24,0,⋯,0,0,0,110,1225,40,4,403,35,167
Pre_5,FS06baseline,5,Pre,P,F,2.356164,106.00,High,82,11,⋯,0,0,50,513,2922,0,0,0,0,0
Pre_6,FS07baseline,6,Pre,A,SU,1.419178,1.32,Low,7,0,⋯,0,0,0,2315,1978,0,16,0,5,28


In [22]:
# setting seed for reproducibility
get_ancombc2_results = function(df, formula, value, paired_test, group_var){
    # add words!!!
    
    set.seed(123)
    # automatically performs a psuedo log transformation
    value2 = value + 1
    output = ancombc2(data = t(df[,value2:ncol(df)]), 
                      meta_data = df[,1:value],
                      tax_level = 'Species', #taxonomic level of interest
                      fix_formula = formula, # specifying the model
                      pairwise = paired_test, # denotes whether the test is paired
                      group = group_var, # independent variable
                      p_adj_method = "fdr") # Holm-Bonferroni adj p values

    # removing the standard error and test statistics from the results
    removed_cols = c('se_', 'W_')
    final_df = output$res %>%
        select(!matches(removed_cols))

    return(final_df)
    }

# calling fn
saliva_raw_model = get_ancombc2_results(baseline_saliva_df, 'Outcome', 8, FALSE, 'Outcome')
saliva_2cov_model = get_ancombc2_results(baseline_saliva_df, 'Age + IgE + Outcome', 8, FALSE, 'Outcome')
stool_raw_model = get_ancombc2_results(baseline_stool_df, 'Outcome', 9, FALSE, 'Outcome')
stool_2cov_model = get_ancombc2_results(baseline_stool_df, 'Age + IgE + Outcome', 9, FALSE, 'Outcome')

Checking the input data type ...

The input data is of type: matrix

The imported data is in a generic 'matrix'/'data.frame' format.

PASS

Checking the sample metadata ...

The specified variables in the formula: Outcome

The available variables in the sample metadata: Original_Sample_ID, Subject_No, Time_Point, Treatment, Outcome, Age, IgE, IgE_Binarized

PASS

Checking other arguments ...

The number of groups of interest is: 2

Warning message:
“The group variable has < 3 categories 
The multi-group comparisons (global/pairwise/dunnet/trend) will be deactivated”
The sample size per group is: F = 27, SU = 10

PASS

Obtaining initial estimates ...

Estimating sample-specific biases ...

Loading required package: foreach


Attaching package: ‘foreach’


The following objects are masked from ‘package:purrr’:

    accumulate, when


Loading required package: rngtools

Warning message in sqrt(sweep(var_hat, 2, var_delta, "*")):
“NaNs produced”
ANCOM-BC2 primary results ...

Conducting se

## Saliva Raw Results

In [23]:
# passed sensitivity filter and is significant 
saliva_raw_model %>%
    filter(diff_robust_OutcomeSU == TRUE) %>%
    nrow()

# passed sensitivity analysis with a relaxed filter
saliva_raw_model %>%
    filter(q_OutcomeSU < 0.1 & passed_ss_OutcomeSU == TRUE) %>%
    nrow()

# significant with a relaxed filter
saliva_raw_model %>%
    filter(q_OutcomeSU < 0.1) %>%
    nrow()

[1] 0

[1] 0

[1] 9

## Saliva Adjusted Results

In [24]:
# passed sensitivity filter and is significant 
saliva_2cov_model %>%
    filter(diff_robust_OutcomeSU == TRUE) %>%
    nrow()

# passed sensitivity analysis with a relaxed filter
saliva_2cov_model %>%
    filter(q_OutcomeSU < 0.1 & passed_ss_OutcomeSU == TRUE) %>%
    nrow()

# significant with a relaxed filter
saliva_2cov_model %>%
    filter(q_OutcomeSU < 0.1) %>%
    nrow()

[1] 0

[1] 8

[1] 8

## Stool Raw Results

In [25]:
# passed sensitivity filter and is significant 
stool_raw_model %>%
    filter(diff_robust_OutcomeSU == TRUE) %>%
    nrow()

# passed sensitivity analysis with a relaxed filter
stool_raw_model %>%
    filter(q_OutcomeSU < 0.1 & passed_ss_OutcomeSU == TRUE) %>%
    nrow()

# significant with a relaxed filter
stool_raw_model %>%
    filter(q_OutcomeSU < 0.1) %>%
    nrow()

[1] 0

[1] 17

[1] 22

## Stool Adjusted Results

In [26]:
# passed sensitivity filter and is significant 
stool_2cov_model %>%
    filter(diff_robust_OutcomeSU == TRUE) %>%
    nrow()

# passed sensitivity analysis with a relaxed filter
stool_2cov_model %>%
    filter(q_OutcomeSU < 0.1 & passed_ss_OutcomeSU == TRUE) %>%
    nrow()

# significant with a relaxed filter
stool_2cov_model %>%
    filter(q_OutcomeSU < 0.1) %>%
    nrow()

[1] 0

[1] 3

[1] 8

# 2. What microbiome species associate with IgE concentrations at baseline? How is that affected after controlling for age?

In [27]:
# first removing 1 participant that didn't have IgE
baseline_saliva_df = baseline_saliva_df %>%
    filter(IgE_Binarized != 'NA')
baseline_stool_df = baseline_stool_df %>%
    filter(IgE_Binarized != 'NA')

In [28]:
# calling fn
saliva_ige = get_ancombc2_results(baseline_saliva_df, 'IgE_Binarized', 8, FALSE, 'IgE_Binarized')
saliva_ige_cov = get_ancombc2_results(baseline_saliva_df, 'Age + IgE_Binarized', 8, FALSE, 'IgE_Binarized')
stool_ige = get_ancombc2_results(baseline_stool_df, 'IgE_Binarized', 9, FALSE, 'IgE_Binarized')
stool_ige_cov = get_ancombc2_results(baseline_stool_df, 'Age + IgE_Binarized', 9, FALSE, 'IgE_Binarized')

Checking the input data type ...

The input data is of type: matrix

The imported data is in a generic 'matrix'/'data.frame' format.

PASS

Checking the sample metadata ...

The specified variables in the formula: IgE_Binarized

The available variables in the sample metadata: Original_Sample_ID, Subject_No, Time_Point, Treatment, Outcome, Age, IgE, IgE_Binarized

PASS

Checking other arguments ...

The number of groups of interest is: 2

Warning message:
“The group variable has < 3 categories 
The multi-group comparisons (global/pairwise/dunnet/trend) will be deactivated”
The sample size per group is: Low = 22, High = 14

PASS

Obtaining initial estimates ...

Estimating sample-specific biases ...

Warning message in sqrt(sweep(var_hat, 2, var_delta, "*")):
“NaNs produced”
ANCOM-BC2 primary results ...

Conducting sensitivity analysis for pseudo-count addition to 0s ...
For taxa that are significant but do not pass the sensitivity analysis,
they are marked in the 'passed_ss' column and

In [31]:
# passed sensitivity filter and is significant 
saliva_ige %>%
    filter(diff_robust_IgE_BinarizedHigh == TRUE) %>%
    nrow()

# passed sensitivity analysis with a relaxed filter
saliva_ige %>%
    filter(q_IgE_BinarizedHigh < 0.1 & passed_ss_IgE_BinarizedHigh == TRUE) %>%
    nrow()

# significant with a relaxed filter
saliva_ige %>%
    filter(q_IgE_BinarizedHigh < 0.1) %>%
    nrow()

[1] 0

[1] 1

[1] 4

In [32]:
# passed sensitivity filter and is significant 
saliva_ige_cov %>%
    filter(diff_robust_IgE_BinarizedHigh == TRUE) %>%
    nrow()

# passed sensitivity analysis with a relaxed filter
saliva_ige_cov %>%
    filter(q_IgE_BinarizedHigh < 0.1 & passed_ss_IgE_BinarizedHigh == TRUE) %>%
    nrow()

# significant with a relaxed filter
saliva_ige_cov %>%
    filter(q_IgE_BinarizedHigh < 0.1) %>%
    nrow()

[1] 0

[1] 7

[1] 8

In [33]:
# passed sensitivity filter and is significant 
stool_ige %>%
    filter(diff_robust_IgE_BinarizedHigh == TRUE) %>%
    nrow()

# passed sensitivity analysis with a relaxed filter
stool_ige %>%
    filter(q_IgE_BinarizedHigh < 0.1 & passed_ss_IgE_BinarizedHigh == TRUE) %>%
    nrow()

# significant with a relaxed filter
stool_ige %>%
    filter(q_IgE_BinarizedHigh < 0.1) %>%
    nrow()

[1] 0

[1] 15

[1] 50

In [34]:
# passed sensitivity filter and is significant 
stool_ige_cov %>%
    filter(diff_robust_IgE_BinarizedHigh == TRUE) %>%
    nrow()

# passed sensitivity analysis with a relaxed filter
stool_ige_cov %>%
    filter(q_IgE_BinarizedHigh < 0.1 & passed_ss_IgE_BinarizedHigh == TRUE) %>%
    nrow()

# significant with a relaxed filter
stool_ige_cov %>%
    filter(q_IgE_BinarizedHigh < 0.1) %>%
    nrow()

[1] 0

[1] 24

[1] 36

# 3. Are there significant changes in the microbiome across outcome groups? Are those changes impacting the outcome using the treatment, age, and IgE as covariates?

In [42]:
# getting dfs first
paired_saliva_df = filter2_saliva_df %>%
    # removing 12 participants that don't have pre and post samples
    filter(Subject_No %notin% c(1,7,12,13,17,20,21,24,32,33,38,40),
    # retaining those given treatment
          Treatment == 'A') %>%
    # ended up reassinging 'DS' to 'F' due to small sample size
    mutate(Outcome = ifelse(Outcome == 'DS', 'F', Outcome)) %>%
    # making the row names the sample ids
    column_to_rownames(var = 'Sample_ID')

paired_stool_df = filter2_stool_df %>%
    # removing 12 participants that don't have pre and post samples
    filter(Subject_No %notin% c(3,10,11,12,14,15,25,27,32,34:36),
    # retaining those given treatment
          Treatment == 'A') %>%
    # ended up reassinging 'DS' to 'F' due to small sample size
    mutate(Outcome = ifelse(Outcome == 'DS', 'F', Outcome)) %>%
    # making the row names the sample ids
    column_to_rownames(var = 'Sample_ID')

paired_saliva_df$Outcome = factor(paired_saliva_df$Outcome, levels = c('F', 'SU'))
paired_stool_df$Outcome = factor(paired_stool_df$Outcome, levels = c('F', 'SU'))

head(paired_saliva_df)

,Original_Sample_ID,Subject_No,Time_Point,Treatment,Outcome,Age,IgE,IgE_Binarized,k__Bacteria.__.__.__.__.__.__,k__Bacteria.p__Absconditabacteria_.SR1..c__Absconditabacteria_.SR1._.C.1..o__Absconditabacteria_.SR1._.O.1..f__Absconditabacteria_.SR1._.F.1..g__Absconditabacteria_.SR1._.G.1..s__bacterium_HMT_874,⋯,k__Bacteria.p__Proteobacteria.c__Gammaproteobacteria.o__Pasteurellales.f__Pasteurellaceae.__.__,k__Bacteria.p__Proteobacteria.c__Gammaproteobacteria.o__Pasteurellales.f__Pasteurellaceae.g__Aggregatibacter.__,k__Bacteria.p__Proteobacteria.c__Gammaproteobacteria.o__Pasteurellales.f__Pasteurellaceae.g__Aggregatibacter.s__aphrophilus,k__Bacteria.p__Proteobacteria.c__Gammaproteobacteria.o__Pasteurellales.f__Pasteurellaceae.g__Haemophilus.__,k__Bacteria.p__Proteobacteria.c__Gammaproteobacteria.o__Pasteurellales.f__Pasteurellaceae.g__Haemophilus.s__parainfluenzae,k__Bacteria.p__Saccharibacteria_.TM7..c__Saccharibacteria_.TM7._.C.1..o__Saccharibacteria_.TM7._.O.1..f__Saccharibacteria_.TM7._.F.1..g__Saccharibacteria_.TM7._.G.1..__,k__Bacteria.p__Saccharibacteria_.TM7..c__Saccharibacteria_.TM7._.C.1..o__Saccharibacteria_.TM7._.O.1..f__Saccharibacteria_.TM7._.F.1..g__Saccharibacteria_.TM7._.G.1..s__bacterium_HMT_347,k__Bacteria.p__Saccharibacteria_.TM7..c__Saccharibacteria_.TM7._.C.1..o__Saccharibacteria_.TM7._.O.1..f__Saccharibacteria_.TM7._.F.1..g__Saccharibacteria_.TM7._.G.1..s__bacterium_HMT_352,k__Bacteria.p__Saccharibacteria_.TM7..c__Saccharibacteria_.TM7._.C.1..o__Saccharibacteria_.TM7._.O.1..f__Saccharibacteria_.TM7._.F.1..g__Saccharibacteria_.TM7._.G.3..s__bacterium_HMT_351,k__Bacteria.p__Saccharibacteria_.TM7..c__Saccharibacteria_.TM7._.C.1..o__Saccharibacteria_.TM7._.O.1..f__Saccharibacteria_.TM7._.F.1..g__Saccharibacteria_.TM7._.G.6..s__bacterium_HMT_870
,<chr>,<dbl>,<chr>,<chr>,<fct>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Post_2,FS0236mo,2,Post,A,F,1.873973,14.20,Low,63,25,⋯,698,0,43,98,1657,0,0,423,146,123
Pre_2,FS02baseline,2,Pre,A,F,1.873973,14.20,Low,0,16,⋯,0,0,46,371,3359,0,0,0,0,112
Post_4,FS0536mo,4,Post,A,F,3.991781,9.59,Low,6,0,⋯,0,17,0,139,2521,0,0,502,39,101
Pre_4,FS05baseline,4,Pre,A,F,3.991781,9.59,Low,24,0,⋯,0,0,0,110,1225,40,4,403,35,167
Post_6,FS0736mo,6,Post,A,SU,1.419178,1.32,Low,2,24,⋯,0,0,0,463,1646,7,0,167,61,437
Pre_6,FS07baseline,6,Pre,A,SU,1.419178,1.32,Low,7,0,⋯,0,0,0,2315,1978,0,16,0,5,28


In [45]:
# calling fn
paired_saliva_raw = get_ancombc2_results(paired_saliva_df, 'Outcome', 8, TRUE, 'Outcome')
paired_saliva_2cov = get_ancombc2_results(paired_saliva_df, 'Age + IgE + Outcome', 8, TRUE, 'Outcome')
paired_stool_raw = get_ancombc2_results(paired_stool_df, 'Outcome', 9, TRUE, 'Outcome')
paired_stool_2cov = get_ancombc2_results(paired_stool_df, 'Age + IgE + Outcome', 9, TRUE, 'Outcome')

Checking the input data type ...

The input data is of type: matrix

The imported data is in a generic 'matrix'/'data.frame' format.

PASS

Checking the sample metadata ...

The specified variables in the formula: Outcome

The available variables in the sample metadata: Original_Sample_ID, Subject_No, Time_Point, Treatment, Outcome, Age, IgE, IgE_Binarized

PASS

Checking other arguments ...

The number of groups of interest is: 2

Warning message:
“The group variable has < 3 categories 
The multi-group comparisons (global/pairwise/dunnet/trend) will be deactivated”
The sample size per group is: F = 16, SU = 18

PASS

Obtaining initial estimates ...

Estimating sample-specific biases ...

ANCOM-BC2 primary results ...

Conducting sensitivity analysis for pseudo-count addition to 0s ...
For taxa that are significant but do not pass the sensitivity analysis,
they are marked in the 'passed_ss' column and will be treated as non-significant in the 'diff_robust' column.
For detailed instruct

## Paired Saliva Raw Results

In [46]:
# passed sensitivity filter and is significant 
paired_saliva_raw %>%
    filter(diff_robust_OutcomeSU == TRUE) %>%
    nrow()

# passed sensitivity analysis with a relaxed filter
paired_saliva_raw %>%
    filter(q_OutcomeSU < 0.1 & passed_ss_OutcomeSU == TRUE) %>%
    nrow()

# significant with a relaxed filter
paired_saliva_raw %>%
    filter(q_OutcomeSU < 0.1) %>%
    nrow()

[1] 0

[1] 7

[1] 8

## Paired Adjusted Saliva Results

In [47]:
# passed sensitivity filter and is significant 
paired_saliva_2cov %>%
    filter(diff_robust_OutcomeSU == TRUE) %>%
    nrow()

# passed sensitivity analysis with a relaxed filter
paired_saliva_2cov %>%
    filter(q_OutcomeSU < 0.1 & passed_ss_OutcomeSU == TRUE) %>%
    nrow()

# significant with a relaxed filter
paired_saliva_2cov %>%
    filter(q_OutcomeSU < 0.1) %>%
    nrow()

[1] 0

[1] 2

[1] 16

## Paired Stool Results

In [48]:
# passed sensitivity filter and is significant 
paired_stool_raw %>%
    filter(diff_robust_OutcomeSU == TRUE) %>%
    nrow()

# passed sensitivity analysis with a relaxed filter
paired_stool_raw %>%
    filter(q_OutcomeSU < 0.1 & passed_ss_OutcomeSU == TRUE) %>%
    nrow()

# significant with a relaxed filter
paired_stool_raw %>%
    filter(q_OutcomeSU < 0.1) %>%
    nrow()

[1] 0

[1] 6

[1] 6

## Paired Stool Adjusted Results

In [49]:
# passed sensitivity filter and is significant 
paired_stool_2cov %>%
    filter(diff_robust_OutcomeSU == TRUE) %>%
    nrow()

# passed sensitivity analysis with a relaxed filter
paired_stool_2cov %>%
    filter(q_OutcomeSU < 0.1 & passed_ss_OutcomeSU == TRUE) %>%
    nrow()

# significant with a relaxed filter
paired_stool_2cov %>%
    filter(q_OutcomeSU < 0.1) %>%
    nrow()

[1] 0

[1] 8

[1] 49

In [25]:
?ancombc2

ancombc2 {ANCOMBC},R Documentation
data,"the input data. The data parameter should be either a matrix, data.frame, phyloseq or a TreeSummarizedExperiment object. Both phyloseq and TreeSummarizedExperiment objects consist of a feature table (microbial count table), a sample metadata table, a taxonomy table (optional), and a phylogenetic tree (optional). If a matrix or data.frame is provided, ensure that the row names of the metadata match the sample names (column names if taxa_are_rows is TRUE, and row names otherwise) in data. if a phyloseq or a TreeSummarizedExperiment is used, this standard has already been enforced. For detailed information, refer to ?phyloseq::phyloseq or ?TreeSummarizedExperiment::TreeSummarizedExperiment. It is recommended to use low taxonomic levels, such as OTU or species level, as the estimation of sampling fractions requires a large number of taxa."
taxa_are_rows,logical. Whether taxa are positioned in the rows of the feature table. Default is TRUE.
assay.type,alias for assay_name.
assay_name,"character. Name of the count table in the data object (only applicable if data object is a (Tree)SummarizedExperiment). Default is ""counts"". See ?SummarizedExperiment::assay for more details."
rank,alias for tax_level.
tax_level,"character. The taxonomic or non taxonomic(rowData) level of interest. The input data can be analyzed at any taxonomic or rowData level without prior agglomeration. Note that tax_level must be a value from taxonomyRanks or rowData, which includes ""Kingdom"", ""Phylum"" ""Class"", ""Order"", ""Family"" ""Genus"" ""Species"" etc. See ?mia::taxonomyRanks for more details. Default is NULL, i.e., do not perform agglomeration, and the ANCOM-BC2 analysis will be performed at the lowest taxonomic level of the input data."
aggregate_data,"The abundance data that has been aggregated to the desired taxonomic level. This parameter is required only when the input data is in matrix or data.frame format. For phyloseq or TreeSummarizedExperiment data, aggregation is performed by specifying the tax_level parameter."
meta_data,"a data.frame containing sample metadata. This parameter is mandatory when the input data is a generic matrix or data.frame. Ensure that the row names of the metadata match the sample names (column names if taxa_are_rows is TRUE, and row names otherwise) in data."
fix_formula,"the character string expresses how the microbial absolute abundances for each taxon depend on the fixed effects in metadata. When specifying the fix_formula, make sure to include the group variable in the formula if it is not NULL."
rand_formula,the character string expresses how the microbial absolute abundances for each taxon depend on the random effects in metadata. ANCOM-BC2 follows the lmerTest package in formulating the random effects. See ?lmerTest::lmer for more details. Default is NULL.
